In [ ]:
# 环境配置 - 安装必要的依赖包
%pip install anthropic python-dotenv httpx --quiet

import anthropic
import os
import httpx
from dotenv import load_dotenv
import json


In [ ]:
# Claude API配置
load_dotenv()  # 加载环境变量

# 配置Claude客户端（包含代理设置）
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/")  # 配置代理
)

# 定义辅助函数用于调用Claude API
def call_claude(prompt, system="", max_tokens=1000, temperature=0.7):
    """
    调用Claude API的辅助函数
    
    参数:
    - prompt: 用户输入的提示词
    - system: 系统提示词
    - max_tokens: 最大token数
    - temperature: 随机性参数（0-1，越高越随机）
    """
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            system=system,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        # 处理响应内容
        if response.content and len(response.content) > 0:
            content = response.content[0]
            if hasattr(content, 'text'):
                return content.text
            else:
                return str(content)
        return "响应为空"
    except Exception as e:
        return f"API调用错误: {str(e)}"

print("✅ Claude API 配置完成！")


In [ ]:
# 示例1: 没有结构的Prompt
prompt_bad = "分析一下我们公司的销售情况"

print("❌ 糟糕的Prompt示例:")
print(f"'{prompt_bad}'")
print("\n问题:")
print("- 任务不明确：'分析'具体指什么？")
print("- 缺少上下文：哪个时间段？哪些产品？")
print("- 输出格式未定义：要报告还是数据？")
print("- 受众不明：给谁看的？")
print("- 风格未确定：正式还是内部讨论？")


In [ ]:
# 示例2: 使用TACOS框架的高质量Prompt
prompt_good = """
**Task**: 作为高级商业分析师，分析我们公司2024年Q3的销售绩效，识别关键趋势和改进机会。

**Audience**: 面向董事会和高级管理层的战略决策报告。

**Context**: 
- 分析范围：2024年7-9月销售数据
- 重点产品线：软件产品A、硬件产品B、服务项目C
- 比较基准：同期去年数据和Q2数据
- 市场环境：竞争对手推出新产品，经济环境不确定

**Output**: 生成结构化的PowerPoint演示要点，包括：
1. 执行摘要（3个关键发现）
2. 销售趋势分析（数据图表描述）
3. 产品线表现对比
4. 风险识别与机会分析
5. 具体行动建议（优先级排序）

**Style**: 
- 语调：专业、客观、数据驱动
- 长度：每部分150-200字
- 格式：简洁的要点形式，易于制作PPT
"""

print("✅ 优秀的TACOS结构化Prompt:")
print(prompt_good)


In [ ]:
# 实际测试TACOS框架的效果（模拟数据）
sample_sales_data = """
2024年Q3销售数据：
- 软件产品A：营收520万，同比增长15%，环比下降8%
- 硬件产品B：营收380万，同比下降12%，环比增长3%  
- 服务项目C：营收290万，同比增长28%，环比增长15%
- 总营收：1190万，同比增长8%，环比增长2%
- 主要客户：企业客户占70%，个人客户占30%
- 地区分布：北上广深占60%，二线城市占40%
"""

# 组合完整的Prompt（包含实际数据）
full_prompt = f"""
{prompt_good}

**数据输入**:
{sample_sales_data}

请基于以上数据和TACOS框架要求，生成分析报告。
"""

print("🔄 正在调用Claude API进行TACOS框架演示...")
print("⏳ 请稍等，正在生成分析报告...")

# 调用Claude API
response = call_claude(full_prompt, max_tokens=2000, temperature=0.3)
print("\n📊 Claude 基于TACOS框架的分析结果:")
print("="*60)
print(response)


In [ ]:
# 高级技巧演示1: XML标签结构化
xml_prompt = """
<context>
你是一名专业的AI产品经理，正在为团队设计AI Agent的核心功能规划。
</context>

<instructions>
基于当前AI Agent发展趋势，设计一个面向企业客户的智能客服Agent的核心功能模块。
</instructions>

<constraints>
- 必须考虑成本效益
- 技术实现难度要适中
- 需要有明确的ROI预期
- 符合数据隐私法规
</constraints>

<output_format>
请按以下JSON格式输出：
{
  "核心功能模块": [
    {
      "模块名称": "",
      "功能描述": "", 
      "技术难度": "低/中/高",
      "预期ROI": "",
      "实现优先级": "P0/P1/P2"
    }
  ],
  "整体架构建议": "",
  "风险评估": ""
}
</output_format>

<examples>
参考但不限于：自然语言处理、知识库检索、多轮对话管理、情感分析等模块。
</examples>
"""

print("🔧 XML结构化Prompt演示:")
print("="*50)
print(xml_prompt)
print("\n🔄 正在调用Claude API...")

# 调用API
xml_response = call_claude(xml_prompt, max_tokens=1500, temperature=0.5)
print("\n📋 Claude基于XML结构的回应:")
print("="*50)
print(xml_response)


In [ ]:
# 高级技巧演示2: Chain-of-Thought (思维链)
cot_prompt = """
<instructions>
你是一名AI安全专家，需要评估一个AI Agent在金融服务场景中的潜在风险。

请使用思维链方法，在<thinking>标签中展示你的分析过程，然后在<answer>标签中给出最终的风险评估报告。
</instructions>

<context>
某银行计划部署一个AI Agent用于：
1. 客户咨询回答（包括账户查询、产品介绍）
2. 初步的投资建议生成
3. 风险评估和合规检查提醒
4. 异常交易模式识别

该Agent将：
- 接入客户个人信息数据库
- 连接实时交易系统
- 使用第三方大语言模型API
- 24/7无人值守运行
</context>

<output_format>
<thinking>
[在此展示你的逐步分析过程]
</thinking>

<answer>
风险评估报告：
1. 高风险项：
2. 中风险项：
3. 低风险项：
4. 建议的风险缓解策略：
5. 优先级建议：
</answer>
</output_format>
"""

print("🧠 Chain-of-Thought演示:")
print("="*50)
print(cot_prompt)
print("\n🔄 正在调用Claude API进行思维链分析...")

# 调用API（使用较低的temperature确保分析严谨）
cot_response = call_claude(cot_prompt, max_tokens=2000, temperature=0.2)
print("\n🎯 Claude的思维链分析:")
print("="*50)
print(cot_response)


In [ ]:
# 高级技巧演示3: Few-Shot Prompting (多样本提示)
few_shot_prompt = """
我需要你帮助我生成AI Agent功能需求的标准化描述。请参考以下格式：

<examples>
示例1:
功能名称: 智能邮件分类
输入描述: 接收用户邮箱中的未读邮件
处理逻辑: 基于邮件内容、发送者、主题等信息进行分类
输出结果: 将邮件标记为"工作"、"个人"、"营销"、"垃圾邮件"四类
成功指标: 分类准确率>90%，处理速度<2秒/封

示例2:  
功能名称: 会议纪要生成
输入描述: 会议录音文件和参会人员名单
处理逻辑: 语音转文字后提取关键讨论点、决策和行动项
输出结果: 结构化的会议纪要文档，包含议题、决策、待办事项
成功指标: 关键信息提取完整度>95%，生成时间<会议时长的10%
</examples>

现在请为以下AI Agent功能生成标准化描述：

功能概述: 一个能够自动回复客户咨询的智能客服Agent，主要处理售前产品咨询、订单查询、售后支持等场景。

请按照上述格式生成详细的功能需求描述。
"""

print("📝 Few-Shot Prompting演示:")
print("="*50)
print(few_shot_prompt)
print("\n🔄 正在调用Claude API...")

# 调用API
few_shot_response = call_claude(few_shot_prompt, max_tokens=1000, temperature=0.4)
print("\n📋 Claude基于示例的输出:")
print("="*50)
print(few_shot_response)


In [ ]:
# 迭代优化实战演示
print("🔄 Prompt迭代优化实战演示")
print("="*60)

# 第一阶段：基础版本（V1.0）
prompt_v1 = "帮我写一个AI Agent的产品介绍"

print("📝 V1.0 - 基础版本:")
print(f"Prompt: {prompt_v1}")

# 第二阶段：应用TACOS框架（V2.0）
prompt_v2 = """
**Task**: 作为产品市场经理，撰写一个企业级AI Agent产品的市场介绍材料

**Audience**: 面向中大型企业的CTO和IT决策者

**Context**: 
- 产品定位：企业内部效率提升工具
- 主要竞品：Microsoft Copilot、Google Duet AI
- 差异化优势：更好的私有化部署能力和定制化程度

**Output**: 生成包含以下结构的产品介绍：
1. 产品概述（50字）
2. 核心价值主张（3个要点）
3. 技术优势（3个特性）
4. 典型应用场景（2个案例）
5. 部署方案（简要说明）

**Style**: 专业、简洁、技术导向，避免过度营销语言
"""

print(f"\n📋 V2.0 - TACOS框架版本:")
print(prompt_v2)

# 第三阶段：XML优化版本（V3.0）
prompt_v3 = """
<context>
你是一名专业的B2B产品市场经理，专门负责AI/ML产品的市场推广。目标受众是中大型企业的技术决策者。
</context>

<instructions>
撰写一份企业级AI Agent产品的专业介绍材料，突出技术实力和商业价值。
</instructions>

<background>
产品信息：
- 名称：Enterprise AI Agent Platform
- 定位：企业内部智能化效率提升平台
- 核心差异：私有化部署 + 深度定制能力
- 竞品对比：比Microsoft Copilot更灵活，比开源方案更稳定
</background>

<output_format>
请按以下结构输出：

## 产品概述
[50字精炼描述]

## 核心价值主张
1. [价值点1] - [具体说明]
2. [价值点2] - [具体说明]  
3. [价值点3] - [具体说明]

## 技术优势
### [技术特性1]
[详细说明及业务影响]

### [技术特性2]  
[详细说明及业务影响]

### [技术特性3]
[详细说明及业务影响]

## 典型应用场景
### 场景1: [场景名称]
- 应用描述：
- 效果量化：

### 场景2: [场景名称]
- 应用描述：
- 效果量化：

## 部署方案
[私有化部署的技术要点和实施建议]
</output_format>

<constraints>
- 避免过度营销化的语言
- 每个技术优势都要有具体的业务价值体现
- 应用场景必须包含可量化的效果预期
- 总字数控制在800-1000字
</constraints>
"""

print(f"\n🎯 V3.0 - XML结构化优化版本:")
print(prompt_v3)

print(f"\n🔄 正在测试V3.0版本效果...")
v3_response = call_claude(prompt_v3, max_tokens=1500, temperature=0.4)
print(f"\n📊 V3.0版本输出结果:")
print("="*50)
print(v3_response)
